In [1]:
import json
import time
import numpy as np
import pandas as pd


In [2]:
fname = "./goodreads_interactions.csv"
start = time.time()
interactions = pd.read_csv(fname)

interactions = interactions.rename(columns={"book_id":"book_id_csv"})
print(time.time() - start)

106.21423196792603


In [3]:
interactions_book_ids = set(interactions["book_id_csv"].unique())

print(min(interactions_book_ids))
print(max(interactions_book_ids))
print(interactions.shape)
interactions = interactions[(interactions.rating > 0)]
interactions = interactions.drop(['is_reviewed', 'is_read'], axis=1)

0
2360649
(228648342, 5)


,user_id,book_id_csv,rating
0,0,948,5
1,0,947,5
2,0,946,5
3,0,945,5
4,0,944,5
...,...,...,...
228648336,876144,367428,3
228648338,876144,23847,4
228648339,876144,23950,3
228648340,876144,374106,5


In [39]:
fname = './goodreads_books.json'
from io import StringIO
from csv import writer 

output = StringIO()
csv_writer = writer(output)
i = 0
bad = []
start = time.time()
f = open(fname)
for line in f:
    try:
        s = json.loads(line)
    except json.JSONDecodeError:
        bad.append(line)
        continue
#     if(i % 100000 == 0):
#         print(i)
    book_id = s["book_id"]
    title = s['title']
    authors = s['authors']
    average_rating = s['average_rating']
    image_url = s['image_url']
    num_pages = s['num_pages']
    publisher = s['publisher']
    publication_year = s['publication_year']
    ratings_count = s['ratings_count']
    language_code = s['language_code']
    csv_writer.writerow([book_id, title, authors, language_code, average_rating, image_url, num_pages, publisher, publication_year, ratings_count])
    i = i + 1

output.seek(0) # we need to get back to the start of the BytesIO
books = pd.read_csv(output, header=None)
books.columns = ['book_id', 'title', 'authors', 'language_code', 'average_rating', 'image_url', 'num_pages', 'publisher', 'publication_year', 'ratings_count']
f.close()
books.to_csv("books.csv")
print(time.time() - start)


151.7220070362091


In [38]:
fname = 'goodreads_book_genres_initial.json'

output = StringIO()
csv_writer = writer(output)
i = 0
bad = []
start = time.time()
f = open(fname)
for line in f:
    try:
        s = json.loads(line)
    except json.JSONDecodeError:
        bad.append(line)
        continue
#     if(i % 100000 == 0):
#         print(i)
    book_id = s["book_id"]
    genres = s['genres']
    
    csv_writer.writerow([book_id, genres])
    i = i + 1

output.seek(0) # we need to get back to the start of the BytesIO
genres = pd.read_csv(output, header=None)
genres.columns = ['book_id', 'genres']
f.close()
genres.to_csv("genres.csv")
print(time.time() - start)


20.62133026123047


In [45]:
genres.head()
import re
genres['genres_full'] = genres['genres'].apply(lambda x: re.findall("'([^']*)'", x)) 
def splitf(item):
    lst1 = []
    for i in item:
        lst1.append(i.split(","))
    flat_list = [item for sublist in lst1 for item in sublist]
    stripped = [s.strip() for s in flat_list]
    return stripped

genres['genres_full'] = genres['genres_full'].apply(lambda x: splitf(x))
genres['genres_full'] = [','.join(map(str, l)) for l in genres['genres_full']]


In [46]:
genres


,book_id,genres,genres_full
0,5333265,"{'history, historical fiction, biography': 1}","history,historical fiction,biography"
1,1333909,"{'fiction': 219, 'history, historical fiction,...","fiction,history,historical fiction,biography"
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys...","fantasy,paranormal,fiction,mystery,thriller,cr..."
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri...","fiction,romance,mystery,thriller,crime"
4,287140,{'non-fiction': 3},non-fiction
...,...,...,...
2360650,3084038,"{'non-fiction': 5, 'history, historical fictio...","non-fiction,history,historical fiction,biography"
2360651,26168430,"{'mystery, thriller, crime': 4, 'children': 1,...","mystery,thriller,crime,children,fiction"
2360652,2342551,"{'poetry': 14, 'children': 7, 'young-adult': 1...","poetry,children,young-adult,non-fiction"
2360653,22017381,"{'romance': 13, 'mystery, thriller, crime': 2}","romance,mystery,thriller,crime"


In [47]:
book_genres = pd.merge(books, genres, on="book_id")

In [52]:
book_genres = book_genres[['book_id','average_rating', 'ratings_count', 'genres_full']]

In [54]:
book_genres.to_csv('book_genres.csv')

In [5]:
fname = "./book_id_map.csv"
start = time.time()
book_id_map = pd.read_csv(fname)
print(time.time() - start)

0.5779259204864502


In [41]:
books_book_ids = set(books["book_id"].unique())
print(min(books_book_ids))
print(max(books_book_ids))
print(len(books_book_ids))
book_id_map.head()

1
36530431
2360655


,book_id_csv,book_id
0,0,34684622
1,1,34536488
2,2,34017076
3,3,71730
4,4,30422361


In [43]:
books_genres_ids = set(genres["book_id"].unique())
print(min(books_genres_ids))
print(max(books_genres_ids))
print(len(books_genres_ids))
genres.head()

1
36530431
2360655


,book_id,genres
0,5333265,"{'history, historical fiction, biography': 1}"
1,1333909,"{'fiction': 219, 'history, historical fiction,..."
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri..."
4,287140,{'non-fiction': 3}


In [36]:
interactions_book_ids = set(interactions["book_id_csv"].unique())
print(min(interactions_book_ids))
print(max(interactions_book_ids))
print(len(interactions_book_ids))
interactions.head()

0
2360649
2325541


,user_id,book_id_csv,is_read,rating,is_reviewed
0,0,948,1,5,0
1,0,947,1,5,1
2,0,946,1,5,0
3,0,945,1,5,0
4,0,944,1,5,0


In [63]:
book_id_map_id_csv = set(book_id_map["book_id_csv"].unique())
book_id_map_id = set(book_id_map["book_id"].unique())
print(min(book_id_map_id_csv))
print(max(book_id_map_id_csv))
print(len(book_id_map_id_csv))
print(min(book_id_map_id))
print(max(book_id_map_id))
print(len(book_id_map_id))
book_id_map.head()

0
2360649
2360650
1
36530431
2360650


,book_id_csv,book_id
0,0,34684622
1,1,34536488
2,2,34017076
3,3,71730
4,4,30422361


In [7]:
interactions1 = pd.merge(interactions, book_id_map, on="book_id_csv")
print(len(interactions1))

104551549


In [9]:
interactions1 = interactions1.drop(['is_reviewed', 'is_read', 'book_id_csv'], axis=1)
interactions1.head()

,user_id,rating,book_id
0,0,5,12
1,201,3,12
2,339,3,12
3,683,5,12
4,1872,3,12


In [10]:
# ratings_full includes all rows for ratings > 0 even if is_reviewd flag is set to 0
interactions1.to_csv("ratings_full.csv")

In [77]:
book_id_map[book_id_map['book_id_csv'] == 947]

,book_id_csv,book_id
947,947,21


In [40]:
print(len(interactions))
print(len(interactions1))
interactions1
interactions.head()

104551549
104551549


,user_id,book_id_csv,is_read,rating,is_reviewed
0,0,948,1,5,0
1,0,947,1,5,1
2,0,946,1,5,0
3,0,945,1,5,0
4,0,944,1,5,0
